In [50]:
import streamlit as st
import pandas as pd
import numpy as np
import datetime
import os
import plotly.graph_objects as go
import plotly.express as px
import base64
from io import BytesIO
from PIL import Image


from modules.get_scoreboard_name import get_scoreboard_name
from data.raw.club_mapping import club_mapping

# Page config
st.set_page_config(page_title="Czech Football Clubs ELO", layout="wide")

def encode_image(image_path):
    with open(image_path, "rb") as img_file:
        return "data:image/png;base64," + base64.b64encode(img_file.read()).decode()

    
today = datetime.datetime.today()
club_average_opponent_elo = pd.read_csv("data/processed/club_average_opponent_elo.csv")
club_average_opponent_elo["scoreboard"] = club_average_opponent_elo["club_name"].apply(lambda x: get_scoreboard_name(x, club_mapping))
fixtures = pd.read_csv("data/processed/fixtures.csv")
club_elo = pd.read_csv("data/processed/club_elo.csv")


2025-03-01 17:43:09.895 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
club_average_opponent_elo = club_average_opponent_elo.sort_values(by="average_opponent_elo")

# Create figure
fig = go.Figure()

# Add the club's own ELO rating (wider, gray bars)
fig.add_trace(go.Bar(
    y=club_average_opponent_elo["club_name"],
    x=club_average_opponent_elo["club_elo"],
    name="Club ELO",
    orientation='h',
    marker=dict(color='rgba(0,0,0,0.4)'),
    width=0.7  # Wider bars
))

# Add the average opponent ELO (thinner, colored bars)
fig.add_trace(go.Bar(
    y=club_average_opponent_elo["club_name"],
    x=club_average_opponent_elo["average_opponent_elo"],
    name="Average Opponent ELO",
    orientation='h',
    marker=dict(color='blue'),
    width=0.5,  # Thinner bars
    text=club_average_opponent_elo["average_opponent_elo"].astype(int),
    textposition='inside'
))

# Add club logos
for idx, row in club_average_opponent_elo.iterrows():
    club_info = next((item for item in club_mapping.values() if item["club_name"] == row["club_name"]), None)
    if club_info:
        fig.add_layout_image(
            dict(
                source=Image.open(club_info["club_logo"]),
                xref="paper", yref="y",
                x=0, y=row["club_name"],
                sizex=0.1, sizey=1,
                xanchor="right", yanchor="middle"
            )
        )

# Layout adjustments
fig.update_layout(
    barmode='overlay',  # Overlapping bars
    width=300,  # Adjust width as needed
    height=600,  # Adjust height as needed
    yaxis=dict(
        tickmode='array',
        tickvals=club_average_opponent_elo["club_name"],
        ticktext=club_average_opponent_elo["scoreboard"],
        tickfont=dict(size=10),
        tickangle=0,
        automargin=True,
        ticklabelposition="outside",
        ticklen=20,
        tickcolor='white',
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.2,
        xanchor="center",
        x=0.5
    ),
    margin=dict(l=100, r=20, t=20, b=100)  # Adjust margins as needed
)

fig.show()